In [1]:
import scrapy
from scrapy.crawler import CrawlerProcess
import json
from scrapy.item import Item, Field
import time
import random

# Create spider
class movies_spider(scrapy.Spider):

    # Assign name to spider
    name = "movies_spider"

    start_urls = [
        "https://en.wikipedia.org/wiki/List_of_Academy_Award-winning_films"
    ]
    
    # Output
    custom_settings = {
        'FEEDS': { 'data_wikiM.json': {'format': 'json', 'overwrite': True, 'encofing':'utf-8'}}
        }

          
    # Extract film info from first URL        
    def parse(self, response):
        for name in response.xpath('//table[@class="wikitable sortable"]//tr'):
            time.sleep(random.uniform(0.20, 0.70))
            # Was this movie the Best Film winner?
            if name.xpath('.//b').get() is not None:
                best_movie = True
            else:
                best_movie = False
            yield {
               'title': name.xpath('.//i//text()').get(),
               'year': name.xpath('(.//a)[2]/text()').get(),
               'best movie': best_movie,
               'awards': name.xpath('(./td)[3]/text()').get(),
               'nominations': name.xpath('(./td)[4]/text()').get(),
               'linkMovie': str(name.xpath('.//i//a/@href').get())
            }   
        
        # Iterate over each movie link
        for item in response.xpath('//table[@class = "wikitable sortable"]//i//a/@href'):
            time.sleep(random.uniform(0.20, 0.70))
            if item is not None:
                yield response.follow(item.get(), callback=self.parse_movies)
                
            
    def parse_movies(self, response):
        # Is this a documentary film?
        if response.xpath('//div[@class="catlinks"]') is None:
            documentary = None  
        elif response.xpath('//div[@class="catlinks"]//a[contains(text(), "documentary") or contains(text(), "Documentary")]').get() is not None:
            documentary = True
        else:
            documentary = False
        
        # Dict with movie name + link + documentary
        dict_movie = {'movie': response.xpath('//h1[@id="firstHeading"]/i/text()').get(),
                     'link': response.request.url, 
                      'documentary': documentary
                     }
        
                # Information we'll get
        response_type = ['Directed', 'Screenplay', 
                         'Starring', 'Produced', 
                         'Cinematography', 'Edited',
                         'Music', 'Production',
                         'Distributed', 'Release date',
                         'Running time', 'Countr',
                         'Language', 'Budget',
                         'Box office'
                        ]
        
        # Gather all response_type info in URL
        for tipo in response_type:
            path_template = '//table[@class="infobox vevent"]//th[contains(., "{}")]/following-sibling::td'.format(tipo)
            crew = []
            if response.xpath(path_template+'//ul').get() is not None:
                path_template = path_template+'//li'
            for name in response.xpath(path_template):
                crew.append(name.xpath('.//text()').get())
            dict_movie[tipo] = crew
            time.sleep(random.uniform(0.20, 0.70))

        # Yield full dictionary   
        yield dict_movie

        # Get all available URLs for the cast
        path_template = '//table[@class="infobox vevent"]//th[contains(text(), "Starring")]/following-sibling::td'
        if response.xpath(path_template+'//ul').get() is not None:
            path_template = path_template+'//li'
        
        for item in response.xpath(path_template+'//@href'):
            if item is not None:
                yield response.follow(item.get(), callback=self.parse_cast)
                time.sleep(random.uniform(0.20, 0.70))
                
            
     # Get cast details
    def parse_cast(self, response):
        time.sleep(random.uniform(0.20, 0.70))
        
        # Is this person a woman?
        females_sum = 0
        males_sum = 0
        female_list = ['" her "', '"Her "', 
                       '" she "', '"She "', 
                       '" female "', 
                       '" actress "', 
                       '" woman "']
        male_list = ['" his "', '"His "', 
                     '" he "', '"He "', 
                     '" male "', 
                     '" man "']
        
        if response.xpath('//div[@class="catlinks"]//a[contains(text(), "women") or contains(text(), "actresses") or contains(text(), "females")]').get() is not None:
            female = True
        elif response.xpath('//div[@class="catlinks"]//a[contains(text(), "men") or contains(text(), "actors") or contains(text(), "males")]').get() is not None:
            female = False 
        else:
            for e in response.xpath('//p'):
                for word_fem in female_list:
                    path = 'count(.//text()[contains(.,{})])'.format(word_fem)
                    females_sum = females_sum + float(e.xpath(path).get())
                    
                for word_male in male_list:
                    path = 'count(.//text()[contains(., {})])'.format(word_male)
                    males_sum = males_sum + float(e.xpath(path).get())

            if females_sum > males_sum:
                female = True
            elif females_sum < males_sum:
                female = False
            else:
                female = 'NA'
            
        yield {'name': response.xpath('//h1[@id="firstHeading"]//text()').get(),
               'female': female,
              'birthdate': response.xpath('//span[@class="bday"]/text()').get(),
               'birthplace': response.xpath('//div[@class="birthplace"]//text()').getall(),
               'link': response.request.url
              }
                     
            
if __name__ == "__main__":
# Crawler creation
  process = CrawlerProcess({'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)',
                    'DOWNLOAD_HANDLERS': {'s3': None},
                    'LOG_ENABLED': True
                })

# Crawler star
process.crawl(movies_spider)

# Launch spider
process.start()


2022-11-16 18:45:40 [scrapy.utils.log] INFO: Scrapy 2.7.0 started (bot: scrapybot)
2022-11-16 18:45:40 [scrapy.utils.log] INFO: Versions: lxml 4.6.3.0, libxml2 2.9.10, cssselect 1.2.0, parsel 1.6.0, w3lib 2.0.1, Twisted 22.8.0, Python 3.8.8 (default, Apr 13 2021, 15:08:03) [MSC v.1916 64 bit (AMD64)], pyOpenSSL 22.1.0 (OpenSSL 3.0.5 5 Jul 2022), cryptography 38.0.1, Platform Windows-10-10.0.22621-SP0
2022-11-16 18:45:40 [scrapy.crawler] INFO: Overridden settings:
{'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)'}
2022-11-16 18:45:40 [py.warnings] WARNING: C:\Users\vicju\anaconda3\lib\site-packages\scrapy\utils\request.py:231: ScrapyDeprecationWarning: '2.6' is a deprecated value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting.

It is also the default value. In other words, it is normal to get this warning if you have not defined a value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting. This is so for backward compatibility reasons, but it will change in a

2022-11-16 18:45:49 [scrapy.core.scraper] DEBUG: Scraped from <200 https://en.wikipedia.org/wiki/List_of_Academy_Award-winning_films>
{'title': 'Cruella', 'year': '2021', 'best movie': False, 'awards': '1', 'nominations': '2\n', 'linkMovie': '/wiki/Cruella_(film)'}
2022-11-16 18:45:50 [scrapy.core.scraper] DEBUG: Scraped from <200 https://en.wikipedia.org/wiki/List_of_Academy_Award-winning_films>
{'title': 'Nomadland', 'year': '2020', 'best movie': True, 'awards': '3', 'nominations': '6\n', 'linkMovie': '/wiki/Nomadland_(film)'}
2022-11-16 18:45:51 [scrapy.core.scraper] DEBUG: Scraped from <200 https://en.wikipedia.org/wiki/List_of_Academy_Award-winning_films>
{'title': 'The Father', 'year': '2020', 'best movie': False, 'awards': '2', 'nominations': '6\n', 'linkMovie': '/wiki/The_Father_(2020_film)'}
2022-11-16 18:45:51 [scrapy.core.scraper] DEBUG: Scraped from <200 https://en.wikipedia.org/wiki/List_of_Academy_Award-winning_films>
{'title': 'Judas and the Black Messiah', 'year': '2020

2022-11-16 18:46:04 [scrapy.core.scraper] DEBUG: Scraped from <200 https://en.wikipedia.org/wiki/List_of_Academy_Award-winning_films>
{'title': 'American Factory', 'year': '2019', 'best movie': False, 'awards': '1', 'nominations': '1\n', 'linkMovie': '/wiki/American_Factory'}
2022-11-16 18:46:04 [scrapy.core.scraper] DEBUG: Scraped from <200 https://en.wikipedia.org/wiki/List_of_Academy_Award-winning_films>
{'title': 'Hair Love', 'year': '2019', 'best movie': False, 'awards': '1\n', 'nominations': '1\n', 'linkMovie': '/wiki/Hair_Love'}
2022-11-16 18:46:04 [scrapy.core.scraper] DEBUG: Scraped from <200 https://en.wikipedia.org/wiki/List_of_Academy_Award-winning_films>
{'title': 'Green Book', 'year': '2018', 'best movie': True, 'awards': '3', 'nominations': '5\n', 'linkMovie': '/wiki/Green_Book_(film)'}
2022-11-16 18:46:05 [scrapy.core.scraper] DEBUG: Scraped from <200 https://en.wikipedia.org/wiki/List_of_Academy_Award-winning_films>
{'title': 'Bohemian Rhapsody', 'year': '2018', 'best 

2022-11-16 18:46:16 [scrapy.core.scraper] DEBUG: Scraped from <200 https://en.wikipedia.org/wiki/List_of_Academy_Award-winning_films>
{'title': 'Icarus', 'year': '2017', 'best movie': False, 'awards': '1', 'nominations': '1\n', 'linkMovie': '/wiki/Icarus_(2017_film)'}
2022-11-16 18:46:17 [scrapy.core.scraper] DEBUG: Scraped from <200 https://en.wikipedia.org/wiki/List_of_Academy_Award-winning_films>
{'title': 'The Silent Child', 'year': '2017', 'best movie': False, 'awards': '1', 'nominations': '1\n', 'linkMovie': '/wiki/The_Silent_Child'}
2022-11-16 18:46:17 [scrapy.core.scraper] DEBUG: Scraped from <200 https://en.wikipedia.org/wiki/List_of_Academy_Award-winning_films>
{'title': 'Flesh and Sand', 'year': '2017', 'best movie': False, 'awards': '0 (1)', 'nominations': '0\n', 'linkMovie': '/wiki/Flesh_and_Sand'}
2022-11-16 18:46:18 [scrapy.core.scraper] DEBUG: Scraped from <200 https://en.wikipedia.org/wiki/List_of_Academy_Award-winning_films>
{'title': 'Moonlight', 'year': '2016', 'bes

2022-11-16 18:46:30 [scrapy.core.scraper] DEBUG: Scraped from <200 https://en.wikipedia.org/wiki/List_of_Academy_Award-winning_films>
{'title': 'A Girl in the River: The Price of Forgiveness', 'year': '2015', 'best movie': False, 'awards': '1', 'nominations': '1\n', 'linkMovie': '/wiki/A_Girl_in_the_River:_The_Price_of_Forgiveness'}
2022-11-16 18:46:30 [scrapy.core.scraper] DEBUG: Scraped from <200 https://en.wikipedia.org/wiki/List_of_Academy_Award-winning_films>
{'title': 'Son of Saul', 'year': '2015', 'best movie': False, 'awards': '1', 'nominations': '1\n', 'linkMovie': '/wiki/Son_of_Saul'}
2022-11-16 18:46:31 [scrapy.core.scraper] DEBUG: Scraped from <200 https://en.wikipedia.org/wiki/List_of_Academy_Award-winning_films>
{'title': 'Spectre', 'year': '2015', 'best movie': False, 'awards': '1', 'nominations': '1\n', 'linkMovie': '/wiki/Spectre_(2015_film)'}
2022-11-16 18:46:31 [scrapy.core.scraper] DEBUG: Scraped from <200 https://en.wikipedia.org/wiki/List_of_Academy_Award-winning_

...

2022-11-16 22:47:34 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://en.wikipedia.org/wiki/Wings_(1927_film)> (referer: https://en.wikipedia.org/wiki/List_of_Academy_Award-winning_films)
2022-11-16 22:47:42 [scrapy.core.scraper] DEBUG: Scraped from <200 https://en.wikipedia.org/wiki/Who%27s_Who_in_Animal_Land>
{'movie': "Who's Who in Animal Land", 'link': 'https://en.wikipedia.org/wiki/Who%27s_Who_in_Animal_Land', 'documentary': False, 'Directed': ['Lou Lilly'], 'Screenplay': [], 'Starring': ['Ken Carpenter'], 'Produced': ['Jerry Fairbanks'], 'Cinematography': [], 'Edited': [], 'Music': [], 'Production': [], 'Distributed': ['Paramount Pictures'], 'Release date': ['December\xa02,\xa01944'], 'Running time': ['10 minutes'], 'Countr': ['United States'], 'Language': ['English'], 'Budget': [], 'Box office': []}
2022-11-16 22:47:43 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://en.wikipedia.org/wiki/Wings_Over_Everest> (referer: https://en.wikipedia.org/wiki/List_of_Academy_Awar

2022-11-16 22:48:30 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://en.wikipedia.org/wiki/Woman_of_the_Year> (referer: https://en.wikipedia.org/wiki/List_of_Academy_Award-winning_films)
2022-11-16 22:48:30 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://en.wikipedia.org/wiki/The_Wizard_of_Oz_(1939_film)> (referer: https://en.wikipedia.org/wiki/List_of_Academy_Award-winning_films)
2022-11-16 22:48:30 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://en.wikipedia.org/wiki/Richard_Arlen> (referer: https://en.wikipedia.org/wiki/Wings_(1927_film))
2022-11-16 22:48:30 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://en.wikipedia.org/wiki/Clara_Bow> (referer: https://en.wikipedia.org/wiki/Wings_(1927_film))
2022-11-16 22:48:36 [scrapy.core.scraper] DEBUG: Scraped from <200 https://en.wikipedia.org/wiki/With_the_Marines_at_Tarawa>
{'movie': 'With the Marines at Tarawa', 'link': 'https://en.wikipedia.org/wiki/With_the_Marines_at_Tarawa', 'documentary': True, 'Directed':

2022-11-16 22:49:26 [scrapy.core.scraper] DEBUG: Scraped from <200 https://en.wikipedia.org/wiki/Richard_E._Byrd>
{'name': 'Richard E. Byrd', 'female': False, 'birthdate': '1888-10-25', 'birthplace': [], 'link': 'https://en.wikipedia.org/wiki/Richard_E._Byrd'}
2022-11-16 22:49:28 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://en.wikipedia.org/wiki/Written_on_the_Wind> (referer: https://en.wikipedia.org/wiki/List_of_Academy_Award-winning_films)
2022-11-16 22:49:28 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://en.wikipedia.org/wiki/Wrestling_Swordfish> (referer: https://en.wikipedia.org/wiki/List_of_Academy_Award-winning_films)
2022-11-16 22:49:30 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://en.wikipedia.org/wiki/Wuthering_Heights_(1939_film)> (referer: https://en.wikipedia.org/wiki/List_of_Academy_Award-winning_films)
2022-11-16 22:49:30 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://en.wikipedia.org/wiki/Vera_Ellen> (referer: https://en.wikipedia.org/

2022-11-16 22:50:17 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://en.wikipedia.org/wiki/Charley_Grapewin> (referer: https://en.wikipedia.org/wiki/The_Wizard_of_Oz_(1939_film))
2022-11-16 22:50:17 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://en.wikipedia.org/wiki/Yesterday,_Today_and_Tomorrow> (referer: https://en.wikipedia.org/wiki/List_of_Academy_Award-winning_films)
2022-11-16 22:50:17 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://en.wikipedia.org/wiki/Oscar_Homolka> (referer: https://en.wikipedia.org/wiki/The_Wonderful_World_of_the_Brothers_Grimm)
2022-11-16 22:50:18 [scrapy.core.scraper] DEBUG: Scraped from <200 https://en.wikipedia.org/wiki/Billie_Burke>
{'name': 'Billie Burke', 'female': True, 'birthdate': '1884-08-07', 'birthplace': ['Washington, D.C., U.S.'], 'link': 'https://en.wikipedia.org/wiki/Billie_Burke'}
2022-11-16 22:50:25 [scrapy.core.scraper] DEBUG: Scraped from <200 https://en.wikipedia.org/wiki/The_Yankee_Doodle_Mouse>
{'movie': 'The Yan

2022-11-16 22:51:07 [scrapy.core.scraper] DEBUG: Scraped from <200 https://en.wikipedia.org/wiki/Claude_Jarman_Jr.>
{'name': 'Claude Jarman Jr.', 'female': False, 'birthdate': '1934-09-27', 'birthplace': ['Nashville, Tennessee', ', U.S.'], 'link': 'https://en.wikipedia.org/wiki/Claude_Jarman_Jr.'}
2022-11-16 22:51:09 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://en.wikipedia.org/wiki/Lila_Kedrova> (referer: https://en.wikipedia.org/wiki/Zorba_the_Greek_(film))
2022-11-16 22:51:09 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://en.wikipedia.org/wiki/Sotiris_Moustakas> (referer: https://en.wikipedia.org/wiki/Zorba_the_Greek_(film))
2022-11-16 22:51:09 [scrapy.core.scraper] DEBUG: Scraped from <200 https://en.wikipedia.org/wiki/Lila_Kedrova>
{'name': 'Lila Kedrova', 'female': True, 'birthdate': '1909-10-09', 'birthplace': ['St. Petersburg', ', ', 'Russian Empire'], 'link': 'https://en.wikipedia.org/wiki/Lila_Kedrova'}
2022-11-16 22:51:10 [scrapy.core.scraper] DEBUG: Scrape